In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange


import flygym.common
import flygym.mujoco
import flygym.mujoco.preprogrammed

from flygym.mujoco import Parameters


from nmf_test import (
    NeuromechflyGrooming,
    plot_state_and_contacts,
)

from nmf_project import (
    NeuromechflyProject,
)

from scipy.signal import find_peaks, medfilt
from IPython.display import Video

from flygym.mujoco.examples.rule_based_controller import PreprogrammedSteps

In [2]:
class NeuromechflyDust(NeuromechflyGrooming):
    def __init__(
        self,
        back,
        sim_params,
        init_dust_level=5.0,
        saturation_level=10.0,
        ant_decay=0.05,
        abdomen_decay=0.01,
        eye_decay=0.01,
        foreleg_decay=0.01,
        hindleg_decay=0.01,
        dusted_areas = [],
        touch_thr=1.0,
    ):
        self.back = back
        if self.back == 0:
            self.dusted_areas = ["R_antenna", "L_antenna", "RF_leg", "LF_leg", "R_eye", "L_eye"]
            self.bodies_in_app = {
                "R_antenna": ["RPedicel", "RArista", "RFuniculus"],
                "L_antenna": ["LPedicel", "LArista", "LFuniculus"],
                "RF_leg": [
                    "RFTarsus1",
                    "RFTarsus2",
                    "RFTarsus3",
                    "RFTarsus4",
                    "RFTarsus5",
                    "RFTibia",
                ],
                "LF_leg": [
                    "LFTarsus1",
                    "LFTarsus2",
                    "LFTarsus3",
                    "LFTarsus4",
                    "LFTarsus5",
                    "LFTibia",
                ],
                "R_eye": ["REye"],
                "L_eye": ["LEye"],
            }
        else:
            self.dusted_areas = ["RH_leg", "LH_leg", "Abdomen"]
            self.bodies_in_app = {
                "RH_leg": [
                    "RHTarsus1",
                    "RHTarsus2",
                    "RHTarsus3",
                    "RHTarsus4",
                    "RHTarsus5",
                    "RHTibia",
                ],
                "LH_leg": [
                    "LHTarsus1",
                    "LHTarsus2",
                    "LHTarsus3",
                    "LHTarsus4",
                    "LHTarsus5",
                    "LHTibia",
                ],
                "Abdomen": [
                    "A1A2",
                    "A3",
                    "A4",
                    "A5",
                            ],
            }

        self.dusted_bodies = [
            body for area in self.dusted_areas for body in self.bodies_in_app[area]
        ]
        print("dusted_bodies", self.dusted_bodies)
        # check if the dust level is a list
        self.saturation_level = saturation_level
        if isinstance(init_dust_level, list):
            self.dust_level = np.array(init_dust_level)
        else:
            self.dust_level = np.ones(len(self.dusted_areas)) * init_dust_level
        self.dust_level = np.clip(self.dust_level, 0.0, self.saturation_level)

        self.touch_thr = touch_thr
        if self.back == 0:
            self.ant_dust_decay = ant_decay
            self.eye_dust_decay = eye_decay
            self.foreleg_dust_decay = foreleg_decay
        else:
            self.hindleg_dust_decay = hindleg_decay
            self.abdomen_dust_decay = abdomen_decay

        touch_sensors_locations = []
        self.touch_sensors_ids = []
        self.appendage_touch_sensors = [[]] * len(self.dusted_areas)
        for i, area in enumerate(self.dusted_areas):
            n_touch_sensors_previous = len(touch_sensors_locations)
            if area in self.bodies_in_app:
                touch_sensors_locations += self.bodies_in_app[area]
            else:
                raise ValueError("Unknown dusted area")
            self.appendage_touch_sensors[i] = np.arange(
                n_touch_sensors_previous, len(touch_sensors_locations)
            )

        self.baseline_rgba = np.array([1.0, 1.0, 1.0, 1.0])
        self.no_dust_color = np.array([0.59, 0.39, 0.12, 1.0])
        self.full_dust_color = np.array([0.0, 1.0, 0.0, 1.0])

        super().__init__(
            sim_params=sim_params,
            groom_collision=True,
            touch_sensor_locations=touch_sensors_locations,
        )

        self.update_segment_aspect()

    def _set_geom_colors(self):
        if self.back == 0:
            self.areas_materials = {}
            self.areas_materials["RF_leg"] = []
            self.areas_materials["LF_leg"] = []
            for type_, specs in self._mujoco_config["appearance"].items():
                
            # If part of the dusted bodies, set the color to the baseline
                dusted_bodies_type_match = [
                    db[2:-1].lower() in type_ for db in self.dusted_bodies
                ]
                if (
                    "antenna" in type_
                    or any(dusted_bodies_type_match)
                    and not "arista" in type_
                ):
                    specs["material"]["rgba"] = self.baseline_rgba
                    if type_ == "eye":
                        specs["texture"] = self._mujoco_config["appearance"]["antenna"][
                            "texture"
                        ]
                        specs["texture"]["rgb1"] = self.baseline_rgba[:3]
                        specs["texture"]["rgb2"] = self.baseline_rgba[:3]
                        specs["texture"]["width"] = 1e6
                        specs["texture"]["height"] = 1e6
                        specs["texture"]["random"] = 1e-5
                    elif specs["texture"] is not None:
                        specs["texture"]["rgb1"] = self.baseline_rgba[:3]
                        specs["texture"]["rgb2"] = self.baseline_rgba[:3]

        else:
        
            self.areas_materials = {}
            self.areas_materials["RH_leg"] = []
            self.areas_materials["LH_leg"] = []

            for type_, specs in self._mujoco_config["appearance"].items():
                # If part of the dusted bodies, set the color to the baseline
                dusted_bodies_type_match = [
                    db[2:-1].lower() in type_ for db in self.dusted_bodies
                ]
                if (
                    "a12345" in type_
                    or any(dusted_bodies_type_match)
                ):
                    specs["material"]["rgba"] = self.baseline_rgba
                    if type_ == "a6":
                        specs["texture"] = self._mujoco_config["appearance"]["a12345"][
                            "texture"
                        ]
                        specs["texture"]["rgb1"] = self.baseline_rgba[:3]
                        specs["texture"]["rgb2"] = self.baseline_rgba[:3]
                        specs["texture"]["width"] = 1e6
                        specs["texture"]["height"] = 1e6
                        specs["texture"]["random"] = 1e-5
                    elif specs["texture"] is not None:
                        specs["texture"]["rgb1"] = self.baseline_rgba[:3]
                        specs["texture"]["rgb2"] = self.baseline_rgba[:3]

        

            # Define texture and material
            if specs["texture"] is not None:
                self.model.asset.add(
                    "texture",
                    name=f"{type_}_texture",
                    builtin=specs["texture"]["builtin"],
                    mark="random",
                    width=specs["texture"]["size"],
                    height=specs["texture"]["size"],
                    random=specs["texture"]["random"],
                    rgb1=specs["texture"]["rgb1"],
                    rgb2=specs["texture"]["rgb2"],
                    markrgb=specs["texture"]["markrgb"],
                )
            self.model.asset.add(
                "material",
                name=f"{type_}_material",
                texture=f"{type_}_texture" if specs["texture"] is not None else None,
                rgba=specs["material"]["rgba"],
                specular=0.0,
                shininess=0.0,
                reflectance=0.0,
                texuniform=True,
            )
            # Apply to geoms
            for segment in specs["apply_to"]:
                geom = self.model.find("geom", segment)
                if geom is None:
                    geom = self.model.find("geom", f"{segment}")
                geom.material = f"{type_}_material"

    def get_observation(self):
        obs = super().get_observation()

        appendage_touch = [
            np.sum(obs["touch_sensors"][touch_sensors])
            for touch_sensors in self.appendage_touch_sensors
        ]

        if self.back == 0:
            R_antenna_contact = np.mean(
                appendage_touch[self.dusted_areas.index("R_antenna")]
            )
            L_antenna_contact = np.mean(
                appendage_touch[self.dusted_areas.index("L_antenna")]
            )
            RF_leg_contact = np.mean(appendage_touch[self.dusted_areas.index("RF_leg")])
            LF_leg_contact = np.mean(appendage_touch[self.dusted_areas.index("LF_leg")])
            R_eye_contact = np.mean(appendage_touch[self.dusted_areas.index("R_eye")])
            L_eye_contact = np.mean(appendage_touch[self.dusted_areas.index("L_eye")])
            Abdomen_contact = 0.0
            RH_leg_contact = 0.0
            LH_leg_contact = 0.0
        
        else:
            R_antenna_contact = 0.0
            L_antenna_contact = 0.0
            RF_leg_contact = 0.0
            LF_leg_contact = 0.0
            R_eye_contact = 0.0
            L_eye_contact = 0.0
            Abdomen_contact = np.mean(
                appendage_touch[self.dusted_areas.index("Abdomen")]
            )
            RH_leg_contact = np.mean(appendage_touch[self.dusted_areas.index("RH_leg")])
            LH_leg_contact = np.mean(appendage_touch[self.dusted_areas.index("LH_leg")])

        if self.back == 0:
            is_Rantenna_grooming = (
                R_antenna_contact > self.touch_thr
                and RF_leg_contact > self.touch_thr
                and LF_leg_contact > self.touch_thr
                and not self.back
            )

            is_Lantenna_grooming = (
                L_antenna_contact > self.touch_thr
                and RF_leg_contact > self.touch_thr
                and LF_leg_contact > self.touch_thr
                and not self.back
            )
            
            is_eye_grooming = (
                R_eye_contact > self.touch_thr
                and RF_leg_contact > self.touch_thr
                and L_eye_contact > self.touch_thr
                and LF_leg_contact > self.touch_thr
                and not self.back
            )

            is_foreleg_grooming = (
                RF_leg_contact > self.touch_thr
                and LF_leg_contact > self.touch_thr
                and not is_Rantenna_grooming
                and not is_Lantenna_grooming
                and not is_eye_grooming
                and not self.back
            )

            is_Abdomen_grooming = self.back

            is_hindleg_grooming = self.back

        else:
            is_Rantenna_grooming = not self.back
            is_Lantenna_grooming = not self.back
            is_eye_grooming = not self.back
            is_foreleg_grooming = not self.back

            is_Abdomen_grooming = (
                Abdomen_contact > self.touch_thr
                and RH_leg_contact > self.touch_thr
                and LH_leg_contact > self.touch_thr
                and self.back
            )

            is_hindleg_grooming = (
                RH_leg_contact > self.touch_thr
                and LH_leg_contact > self.touch_thr
                and not is_Abdomen_grooming
                and self.back
            )

        self.update_dust_level(
            is_foreleg_grooming,
            is_Rantenna_grooming,
            is_Lantenna_grooming,
            is_eye_grooming,
            R_antenna_contact,
            L_antenna_contact,
            RF_leg_contact,
            LF_leg_contact,
            R_eye_contact,
            L_eye_contact,
            is_Abdomen_grooming,
            is_hindleg_grooming,
            Abdomen_contact,
            RH_leg_contact,
            LH_leg_contact,
        )

        self.update_segment_aspect()

        # add curr behavior
        obs["is_foreleg_grooming"] = is_foreleg_grooming
        obs["is_Rantenna_grooming"] = is_Rantenna_grooming
        obs["is_Lantenna_grooming"] = is_Lantenna_grooming
        obs["is_eye_grooming"] = is_eye_grooming
        obs["is_hindleg_grooming"] = is_hindleg_grooming
        obs["is_Abdomen_grooming"] = is_Abdomen_grooming

        assert all(self.dust_level >= 0.0) and all(
            self.dust_level <= self.saturation_level
        ), f"Dust level out of bounds: {self.dust_level}"
        # add dust level
        obs["dust_level"] = self.dust_level

        return obs

    def update_segment_aspect(self):
        # change body rgba depending on dust level
        for app, dl in zip(self.dusted_areas, self.dust_level):
            for body in self.bodies_in_app[app]:
                if not "Arista" in body:
                    norm_dl = (dl.copy() - 0.0) / (self.saturation_level - 0.0)
                    self.physics.named.model.geom_rgba[
                        f"Animat/{body}"
                    ] = self.interp_color(norm_dl)

    def interp_color(self, dust_level):
        return (
            self.no_dust_color
            + (self.full_dust_color - self.no_dust_color) * dust_level
        )

    def update_dust_level(
        self,
        is_foreleg_grooming,
        is_Rantenna_grooming,
        is_Lantenna_grooming,
        is_eye_grooming,
        R_antenna_contact,
        L_antenna_contact,
        RF_leg_contact,
        LF_leg_contact,
        R_eye_contact,
        L_eye_contact,
        is_Abdomen_grooming,
        is_hindleg_grooming,
        Abdomen_contact,
        RH_leg_contact,
        LH_leg_contact,
    ):
        is_saturated = self.dust_level >= self.saturation_level
        saturated_app = [
            self.dusted_areas[i] for i, sat in enumerate(is_saturated) if sat
        ]
        is_zero = self.dust_level <= 0.0
        zero_app = [self.dusted_areas[i] for i, zero in enumerate(is_zero) if zero]
    
        if self.back == 0:

            # update dust level bodies that have no dust should not give dust to the forelegs,
            # when the forelegs are saturated, they can not pick up dust anymore
            if is_Rantenna_grooming and not "R_antenna" in zero_app:
                removed_dust = (
                    np.mean([R_antenna_contact, RF_leg_contact, LF_leg_contact])
                    * self.ant_dust_decay
                )
                removed_dust = self.update_leg_dust_antgroom(
                    saturated_app, removed_dust
                )
                self.dust_level[self.dusted_areas.index("R_antenna")] -= removed_dust
            if is_Lantenna_grooming and not "L_antenna" in zero_app:
                removed_dust = (
                    np.mean([L_antenna_contact, RF_leg_contact, LF_leg_contact])
                    * self.ant_dust_decay
                )
                removed_dust = self.update_leg_dust_antgroom(
                    saturated_app, removed_dust
                )
                self.dust_level[self.dusted_areas.index("L_antenna")] -= removed_dust
            if is_eye_grooming and (not "R_eye" in zero_app or not "L_eye" in zero_app):
                removed_dust = (
                    np.mean([R_eye_contact, L_eye_contact, RF_leg_contact, LF_leg_contact])
                    * self.eye_dust_decay
                )
                if "R_eye" in zero_app:
                    # if added to saturated_app no dust can be added to it
                    saturated_app.append("RF_leg")
                elif "L_eye" in zero_app:
                    saturated_app.append("LF_leg")
                removed_dust = self.update_leg_dust_antgroom(
                    saturated_app, removed_dust
                )
                if not "R_eye" in zero_app:
                    self.dust_level[self.dusted_areas.index("R_eye")] -= removed_dust
                if not "L_eye" in zero_app:
                    self.dust_level[self.dusted_areas.index("L_eye")] -= removed_dust
            if is_foreleg_grooming and (not "RF_leg" in zero_app or not "LF_leg" in zero_app):
                removed_dust = (
                    np.mean([RF_leg_contact, LF_leg_contact]) * self.foreleg_dust_decay
                )
                if "RF_leg" in zero_app and "LF_leg" in zero_app:
                    removed_dust = 0
                elif "RF_leg" in zero_app:
                    self.dust_level[self.dusted_areas.index("LF_leg")] -= removed_dust / 2
                elif "LF_leg" in zero_app:
                    self.dust_level[self.dusted_areas.index("RF_leg")] -= removed_dust / 2
                else:
                    self.dust_level[self.dusted_areas.index("RF_leg")] -= removed_dust / 2
                    self.dust_level[self.dusted_areas.index("LF_leg")] -= removed_dust / 2

        else:
            
            # update dust level bodies that have no dust should not give dust to the forelegs,
            # when the forelegs are saturated, they can not pick up dust anymore
            if is_Abdomen_grooming and not "Abdomen" in zero_app:
                removed_dust = (
                    np.mean([Abdomen_contact, RH_leg_contact, LH_leg_contact])
                    * self.abdomen_dust_decay
                )
                removed_dust = self.update_leg_dust_antgroom(
                    saturated_app, removed_dust
                )
                self.dust_level[self.dusted_areas.index("Abdomen")] -= removed_dust
            
            if is_hindleg_grooming and (not "RH_leg" in zero_app or not "LH_leg" in zero_app):
                removed_dust = (
                    np.mean([RH_leg_contact, LH_leg_contact]) * self.hindleg_dust_decay
                )
                if "RH_leg" in zero_app and "LH_leg" in zero_app:
                    removed_dust = 0
                elif "RH_leg" in zero_app:
                    self.dust_level[self.dusted_areas.index("LH_leg")] -= removed_dust / 2
                elif "LH_leg" in zero_app:
                    self.dust_level[self.dusted_areas.index("RH_leg")] -= removed_dust / 2
                else:
                    self.dust_level[self.dusted_areas.index("RH_leg")] -= removed_dust / 2
                    self.dust_level[self.dusted_areas.index("LH_leg")] -= removed_dust / 2

        self.dust_level = np.clip(self.dust_level, 0.0, self.saturation_level)

    def update_leg_dust_antgroom(self, saturated_app, supposed_removed_dust):
        if self.back == 0:
            if "LF_leg" in saturated_app and "RF_leg" in saturated_app:
                return 0.0
            elif "RF_leg" in saturated_app:
                self.dust_level[self.dusted_areas.index("RF_leg")] += (
                    supposed_removed_dust / 2
                )
                return supposed_removed_dust / 2
            elif "LF_leg" in saturated_app:
                self.dust_level[self.dusted_areas.index("LF_leg")] += (
                    supposed_removed_dust / 2
                )
                return supposed_removed_dust / 2
            else:
                self.dust_level[self.dusted_areas.index("RF_leg")] += (
                    supposed_removed_dust / 2
                )
                self.dust_level[self.dusted_areas.index("LF_leg")] += (
                    supposed_removed_dust / 2
                )
                return supposed_removed_dust / 2
        else:
            if "LH_leg" in saturated_app and "RH_leg" in saturated_app:
                return 0.0
            elif "RH_leg" in saturated_app:
                self.dust_level[self.dusted_areas.index("RH_leg")] += (
                    supposed_removed_dust / 2
                )
                return supposed_removed_dust / 2
            elif "LH_leg" in saturated_app:
                self.dust_level[self.dusted_areas.index("LH_leg")] += (
                    supposed_removed_dust / 2
                )
                return supposed_removed_dust / 2
            else:
                self.dust_level[self.dusted_areas.index("RH_leg")] += (
                    supposed_removed_dust / 2
                )
                self.dust_level[self.dusted_areas.index("LH_leg")] += (
                    supposed_removed_dust / 2
                )
                return supposed_removed_dust / 2
            

In [3]:
# Joint angles according to indices
LF_COXA = 0
LF_COXA_ROLL = 1
LF_COXA_YAW = 2
LF_FEMUR = 3
LF_FEMUR_ROLL = 4
LF_TIBIA = 5
LF_TARSUS = 6

LM_COXA = 7
LM_COXA_ROLL = 8
LM_COXA_YAW = 9
LM_FEMUR = 10
LM_FEMUR_ROLL = 11
LM_TIBIA = 12
LM_TARSUS = 13

LH_COXA = 14
LH_COXA_ROLL = 15
LH_COXA_YAW = 16
LH_FEMUR = 17
LH_FEMUR_ROLL = 18
LH_TIBIA = 19
LH_TARSUS = 20

RF_COXA = 21
RF_COXA_ROLL = 22
RF_COXA_YAW = 23
RF_FEMUR = 24
RF_FEMUR_ROLL = 25
RF_TIBIA = 26
RF_TARSUS = 27

RM_COXA = 28
RM_COXA_ROLL = 29
RM_COXA_YAW = 30
RM_FEMUR = 31
RM_FEMUR_ROLL = 32
RM_TIBIA = 33
RM_TARSUS = 34

RH_COXA = 35
RH_COXA_ROLL = 36
RH_COXA_YAW = 37
RH_FEMUR = 38
RH_FEMUR_ROLL = 39
RH_TIBIA = 40
RH_TARSUS = 41

A1A2 = 42
A3 = 43
A4 = 44
A5 = 45
A6 = 46

HEAD = 47
HEAD_YAW = 48
HEAD_ROLL = 49

L_PEDICEL = 50
L_PEDICEL_YAW = 51
R_PEDICEL = 52
R_PEDICEL_YAW = 53

JOINT_LH_COXA = 3
JOINT_LH_COXA_YAW = 4
JOINT_LH_COXA_ROLL = 5
JOINT_LH_FEMUR = 6
JOINT_LH_FEMUR_ROLL = 7
JOINT_LH_TIBIA = 8
JOINT_LH_TARSUS = 9

JOINT_RH_COXA = 10
JOINT_RH_COXA_YAW = 11
JOINT_RH_COXA_ROLL = 12
JOINT_RH_FEMUR = 13
JOINT_RH_FEMUR_ROLL = 14
JOINT_RH_TIBIA = 15
JOINT_RH_TARSUS = 16

JOINT_L_PEDICEL = 17
JOINT_L_PEDICEL_YAW = 18

JOINT_R_PEDICEL = 19
JOINT_R_PEDICEL_YAW = 20

JOINT_A1A2 = 21
JOINT_A3 = 22
JOINT_A4 = 23
JOINT_A5 = 24
JOINT_A6 = 25

NB_ABD_JOINTS = 5
NB_JOINTS = 54

In [4]:
run_time = 1

sim_params = flygym.mujoco.Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    #actuator_kp=10.0,
    enable_adhesion=True,
    draw_adhesion=False,
    draw_contacts=True,    
    render_camera="Animat/camera_left",
)

actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs

target_num_steps = int(run_time / sim_params.timestep) # 10'000


In [5]:
sim_params = flygym.mujoco.Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    #actuator_kp=10.0,
    enable_adhesion=True,
    draw_adhesion=True,
    draw_contacts=False,
    render_camera="Animat/camera_left",
)
nmf = NeuromechflyDust(sim_params=sim_params,
back = 1,
init_dust_level=5.0,
ant_decay=0.01,
eye_decay=0.015,
foreleg_decay=0.005,
)

preprogrammed_steps = PreprogrammedSteps()

swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

actuated_joints = preprogrammed_steps.dofs_per_leg

standing_action = []

for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1]))
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

standing_action.extend([0]*12) # Add the abdomen actuators at zero

last_position = standing_action

    
standing_action = {'joints': standing_action, "adhesion": np.zeros(len(legs))}

for i in range(int(0.2//nmf.timestep)):
    nmf.step(standing_action)
    nmf.render()

nmf.save_video("./outputs/standing_wo_dust_back.mp4")





dusted_bodies ['RHTarsus1', 'RHTarsus2', 'RHTarsus3', 'RHTarsus4', 'RHTarsus5', 'RHTibia', 'LHTarsus1', 'LHTarsus2', 'LHTarsus3', 'LHTarsus4', 'LHTarsus5', 'LHTibia', 'A1A2', 'A3', 'A4', 'A5']


In [6]:
target_num_steps = 5000
foreleg_ids = np.zeros(target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], -1/4*np.pi, target_num_steps)
hind_swings_ids = np.zeros(target_num_steps)

adhesion_action = np.array([1.0 if leg.endswith("F") else 0.0 for leg in legs])
all_joint_angles = []

for i in trange(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("H"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, hind_swings_ids[i]))
        elif leg.endswith("M"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, middle_stance_ids[i]))
        else:
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i]))
    joint_angles.extend([0]*12) # Add abdomen actuators at zero

    all_joint_angles.append(joint_angles.copy())
    
    action = {'joints': np.array(joint_angles), "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
nmf.save_video("./outputs/b_back.mp4")

# Save the last joint angles
last_joint_angles = all_joint_angles[-1]
all_joint_angles_arr = np.array(all_joint_angles)

100%|██████████| 5000/5000 [00:07<00:00, 696.20it/s]


In [7]:
target_num_steps = 5000
half_nbre_steps = target_num_steps//2

In [8]:
abdomen_ctrl_up = np.zeros((target_num_steps)) # shape (nbre_steps, )
abdomen_ctrl_up[:half_nbre_steps] = np.linspace(0, 0.1, half_nbre_steps)
abdomen_ctrl_up[half_nbre_steps:] = np.ones(target_num_steps-half_nbre_steps)*0.1 # abd is up after 1 seconds and stays up
abd_joint = np.tile(abdomen_ctrl_up, (5, 1)).T

In [9]:

all_joint_angles_2nd = np.zeros([target_num_steps,54]) # shape (nbre_steps, 47)
print(all_joint_angles_2nd.shape)

print(len(last_joint_angles))

for i in range(len(last_joint_angles)):
#### LEFT LEGS ####    
    if i == LF_TIBIA: # LF tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LF_TIBIA], 1/3*np.pi, target_num_steps)
    elif i == LF_TARSUS : # LF tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LF_TARSUS], 0, target_num_steps)

    elif i == LM_TIBIA: # LM tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_TIBIA], 1/2*np.pi, target_num_steps)
    elif i == LM_TARSUS: # LM tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_TARSUS], 0, target_num_steps)
    elif i == LM_FEMUR: # LM FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_FEMUR], -3/7*np.pi, target_num_steps)
    # elif i == LM_FEMUR_ROLL: # LM FEMUR ROLL
    #     all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LM_FEMUR_ROLL], last_joint_angles[LM_FEMUR_ROLL]/2, target_num_steps)

    elif i == LH_FEMUR: # LH FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LH_FEMUR], -2/5*np.pi, target_num_steps)
    elif i == LH_TIBIA: # LH tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LH_TIBIA], 3*np.pi/5, target_num_steps)
    elif i == LH_TARSUS: # LH tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[LH_TARSUS], -np.pi/12, target_num_steps)

#### RIGHT LEGS ####
    elif i == RF_TIBIA: # RF tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RF_TIBIA], 1/3*np.pi, target_num_steps)
    elif i == RF_TARSUS: # RF tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RF_TARSUS], 0, target_num_steps)

    elif i == RM_TIBIA: # RM tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_TIBIA], 1/2*np.pi, target_num_steps)
    elif i == RM_TARSUS: # RM tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_TARSUS], 0, target_num_steps)
    elif i == RM_FEMUR: # LM FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_FEMUR], -3/7*np.pi, target_num_steps)
    # elif i == RM_FEMUR_ROLL: # RM FEMUR ROLL
    #     all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RM_FEMUR_ROLL], -last_joint_angles[RM_FEMUR_ROLL]/2, target_num_steps)
    
    elif i == RH_FEMUR: # RH FEMUR
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RH_FEMUR], -2/5*np.pi, target_num_steps)
    elif i == RH_TIBIA: # RH tibia
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RH_TIBIA], 3*np.pi/5, target_num_steps)
    elif i == RH_TARSUS: # RH tarsus
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[RH_TARSUS], -np.pi/12, target_num_steps)
                    
    else:
        all_joint_angles_2nd[:,i] = np.linspace(last_joint_angles[i], last_joint_angles[i], target_num_steps)

all_joint_angles_2nd[:,41:46] = abd_joint
print(all_joint_angles_2nd.shape)
adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

for i in trange(target_num_steps):
    joint = all_joint_angles_2nd[i,:]
    action = {'joints': joint, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
last_joint_angles_2nd = all_joint_angles_2nd[-1,:]
nmf.save_video("./outputs/c_back.mp4")


(5000, 54)
54
(5000, 54)


100%|██████████| 5000/5000 [00:07<00:00, 634.82it/s]


In [10]:
grooming_module_path = Path("./data/grooming_modules_provided_slow.pkl")
with open(grooming_module_path, "rb") as f:
    grooming_modules = pickle.load(f)

timestep = grooming_modules["timestep"]

target_joint_angles_front = grooming_modules["foreleg"]

seq_length = len(grooming_modules['foreleg'][0])


In [11]:
idx_diff_LR = 7 # index difference between the right and left leg joints
symetric_leg = np.zeros_like(target_joint_angles_front) # contains all the joint angles for front leg grooming symmetrically

for i in range(len(target_joint_angles_front)):
    if i in [RH_COXA, RH_FEMUR, RH_TIBIA, RH_TARSUS]:
        symetric_leg[i,:] = target_joint_angles_front[i-idx_diff_LR,:] # map the right pitch joints to the left ones
    elif i in [RH_COXA_YAW, RH_COXA_ROLL, RH_FEMUR_ROLL]:
        symetric_leg[i,:] = -target_joint_angles_front[i-idx_diff_LR,:] # map the right yaw roll joints to the left ones
    else:
        symetric_leg[i,:] = target_joint_angles_front[i,:]

In [12]:
def delay_sequence(seq, start_row, end_row, delay):
    # Extract rows to be delayed
    shifted_seq = seq.copy()
    rows_to_shift = seq[start_row:end_row+1,:]

    # Extract columns
    new_col = rows_to_shift[:, delay:]
    first_columns = rows_to_shift[:, :delay]

    shifted_columns = np.concatenate([new_col, first_columns], axis=1)
    shifted_seq[start_row:end_row+1,:] = shifted_columns
    return shifted_seq

In [13]:
initial_sequence = symetric_leg.copy()
rep = 5
delay = seq_length//2
repeated_sequence = np.tile(initial_sequence, (1, rep))

delayed_sequence = delay_sequence(repeated_sequence, JOINT_RH_COXA, JOINT_RH_TARSUS, delay)
#transpose delayed sequence
delayed_sequence = np.transpose(delayed_sequence)
print(delayed_sequence.shape)

(2790, 21)


In [14]:
inverse_leg_joint = delayed_sequence.copy()
print(inverse_leg_joint.shape)

inverse_leg_joint[JOINT_LH_COXA:JOINT_RH_COXA,:], inverse_leg_joint[JOINT_RH_COXA:JOINT_L_PEDICEL,:] = inverse_leg_joint[JOINT_RH_COXA:JOINT_L_PEDICEL,:].copy(), inverse_leg_joint[JOINT_LH_COXA:JOINT_RH_COXA,:].copy()

(2790, 21)


In [15]:

abd_gain = 0.3
abd_up_arr = np.zeros([rep*seq_length, 54])

# Set the hind legs to the initial position for grooming
for j in range(54):
    if j == LH_COXA:
        print(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_LH_COXA])
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_LH_COXA], rep*seq_length)
    elif j == RH_COXA:
        print(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_RH_COXA])
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_RH_COXA], rep*seq_length)
    elif j == LH_FEMUR:
        print(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_LH_FEMUR])
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_LH_FEMUR] + 7*np.pi/6, rep*seq_length)
    elif j == RH_FEMUR:
        print(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_RH_FEMUR])
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_RH_FEMUR] + 7*np.pi/6, rep*seq_length)
    elif j == LH_FEMUR_ROLL:
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_LH_FEMUR_ROLL] - np.pi/6, rep*seq_length)
    elif j == RH_FEMUR_ROLL:
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], -delayed_sequence[0,JOINT_RH_FEMUR_ROLL] - np.pi/6, rep*seq_length)
    elif j == LH_TIBIA:
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_LH_TIBIA] - 2*np.pi/3, rep*seq_length)
    elif j == RH_TIBIA:
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_RH_TIBIA] - 2*np.pi/3, rep*seq_length)
    elif j == LH_TARSUS:
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_LH_TARSUS] + np.pi/2, rep*seq_length)
    elif j == RH_TARSUS:
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], delayed_sequence[0,JOINT_RH_TARSUS] + np.pi/2, rep*seq_length)
    elif j == A1A2:
        abd_up_arr[:(rep*seq_length)//2,j] = np.linspace(last_joint_angles_2nd[j], abd_gain, rep*seq_length//2)
        abd_up_arr[(rep*seq_length)//2:,j] = np.ones((rep*seq_length)//2)*(abd_gain)
    elif j == A3:
        abd_up_arr[:(rep*seq_length)//2,j] = np.linspace(last_joint_angles_2nd[j], abd_gain, rep*seq_length//2)
        abd_up_arr[(rep*seq_length)//2:,j] = np.ones((rep*seq_length)//2)*(abd_gain)
    elif j == A4:
        abd_up_arr[:(rep*seq_length)//2,j] = np.linspace(last_joint_angles_2nd[j], abd_gain, rep*seq_length//2)
        abd_up_arr[(rep*seq_length)//2:,j] = np.ones((rep*seq_length)//2)*(abd_gain)
    elif j == A5:
        abd_up_arr[:(rep*seq_length)//2,j] = np.linspace(last_joint_angles_2nd[j], abd_gain, rep*seq_length//2)
        abd_up_arr[(rep*seq_length)//2:,j] = np.ones((rep*seq_length)//2)*(abd_gain)
    elif j == A6:
        abd_up_arr[:(rep*seq_length)//2,j] = np.linspace(last_joint_angles_2nd[j], abd_gain, rep*seq_length//2)
        abd_up_arr[(rep*seq_length)//2:,j] = np.ones((rep*seq_length)//2)*(abd_gain)
    else:
        abd_up_arr[:,j] = np.linspace(last_joint_angles_2nd[j], last_joint_angles_2nd[j], rep*seq_length)
   

joint_pos = np.zeros(54)

for i in trange(rep*seq_length):
    joint_pos = abd_up_arr[i, :]
    action = {'joints': joint_pos, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()

last_abd_up = abd_up_arr[:, -1]

if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "abd_up_front.mp4"

elif sim_params.render_camera == "Animat/camera_left":
    frontleg_video = "abd_up_left.mp4"
elif sim_params.render_camera == "Animat/camera_back":
    frontleg_video = "abd_up_back.mp4"
else:
    frontleg_video = "abd_up.mp4"  
    
nmf.save_video(frontleg_video)

0.4922179642981038 0.4687785916290188
-1.2566370614359172 -2.216906375230204
0.4922179642981038 0.3911700493229354
-1.2566370614359172 -2.2952147198199597


100%|██████████| 2790/2790 [00:05<00:00, 516.15it/s]


In [16]:
target_joint_angles_back = delayed_sequence.copy()
print(target_joint_angles_back.shape)

joint_pos = np.zeros(NB_JOINTS)

# nmf.reset()
for i in trange(rep*seq_length):
    for j in range(NB_JOINTS):
        if j == LH_COXA:
            joint_pos[j] = target_joint_angles_back[i, JOINT_LH_COXA]
        elif j == RH_COXA:
            joint_pos[j] = target_joint_angles_back[i, JOINT_RH_COXA]
        elif j == LH_COXA_ROLL:
            joint_pos[j] = target_joint_angles_back[i, JOINT_LH_COXA_ROLL] - np.pi/6
        elif j == RH_COXA_ROLL:
            joint_pos[j] = target_joint_angles_back[i, JOINT_RH_COXA_ROLL] - np.pi/6
        elif j == LH_FEMUR: # Femur
            joint_pos[j] = target_joint_angles_back[i, JOINT_LH_FEMUR] + 7*np.pi/6
        elif j == RH_FEMUR:
            joint_pos[j] = target_joint_angles_back[i, JOINT_RH_FEMUR] + 7*np.pi/6
        elif j == LH_FEMUR_ROLL:
            joint_pos[j] = target_joint_angles_back[i, JOINT_LH_FEMUR_ROLL] - np.pi/6
        elif j == RH_FEMUR_ROLL:
            joint_pos[j] = target_joint_angles_back[i, JOINT_RH_FEMUR_ROLL] - np.pi/6
        elif j == LH_TIBIA: # Tibia 
            joint_pos[j] = target_joint_angles_back[i, JOINT_LH_TIBIA] - 2*np.pi/3
        elif j == RH_TIBIA:
            joint_pos[j] = target_joint_angles_back[i, JOINT_RH_TIBIA] - 2*np.pi/3
        elif j == LH_TARSUS: # Tarsus
            joint_pos[j] = target_joint_angles_back[i, JOINT_LH_TARSUS] + np.pi/2
        elif j == RH_TARSUS:
            joint_pos[j] = target_joint_angles_back[i, JOINT_RH_TARSUS] + np.pi/2
        elif j == A1A2:
            joint_pos[j] = last_abd_up[A1A2]
        elif j == A3:
            joint_pos[j] = last_abd_up[A3]
        elif j == A4:
            joint_pos[j] = last_abd_up[A4]
        elif j == A5:
            joint_pos[j] = last_abd_up[A5]
        elif j == A6:
            joint_pos[j] = last_abd_up[A6]
        else:
            joint_pos[j] = last_joint_angles_2nd[j]
    
    action = {'joints': joint_pos, "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
        
if sim_params.render_camera == "Animat/camera_front":
    frontleg_video = "hindleg_sym_front_LR.mp4"

elif sim_params.render_camera == "Animat/camera_left":
    frontleg_video = "hindleg_sym_left_LR.mp4"

else:
    frontleg_video = "hindleg_sym_back_LR.mp4"  
    
nmf.save_video(frontleg_video)

(2790, 21)


100%|██████████| 2790/2790 [00:04<00:00, 579.24it/s]
